# TMA Analysis code supporting technote SITCOMTN-067
Craig Lage - 15-Nov-23
Updated 15-Dec523 to use TMAEventMaker code.

This notebook characterizes TMA velocity, acceleration, and jerk. 


## Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)
dayObs = 20231129

# Velocity, acceleration, and jerk limits:
# Limits are [Velocity (deg/sec), Accel (deg/sec^2, Jerk (deg/sec^3)]
azLimits = [10.5, 10.5, 42.0]
elLimits = [5.25, 5.25, 21.0]
azDesign = [7.0, 7.0, 28.0]
elDesign = [3.5, 3.5, 14.0]

# This allows you to mask out certain bad events
badEvents = [92]


In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
from astropy.time import Time, TimeDelta
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

# First, let's get the events

In [ ]:
client = EfdClient("idf_efd")
eventMaker = TMAEventMaker()
events = eventMaker.getEvents(dayObs)
Nslews = 0
Ntracks = 0
for event in events:
    if event.type.name == 'TRACKING':
        Ntracks += 1
    elif event.type.name == 'SLEWING':
        Nslews += 1
print(f"There are {len(events)} events, {Nslews} slewing and {Ntracks} tracking")    

# Now plot the Velocity/Accel/Jerk plots.

In [ ]:
# Plotting the Vel/Accel/Jerk plot
%matplotlib inline
maxAzVels = []
maxAzAccels = []
maxAzJerks = []
maxElVels = []
maxElAccels = []
maxElJerks = []
smoothingFactor = 0.2 # In spline creation
kernelSize = 100 # In convolution
kernel = np.ones(kernelSize) / kernelSize

fig = plt.figure(figsize = (8,8))
for event in events:
    try:
        if event.type.name != 'SLEWING':
            # Only slewing events
            continue
        if event.duration < 0.1:
            # eliminate bogus events
            continue
        if event.seqNum in badEvents:
            # Allows to eliminate bad events
            continue
            
        start = event.begin - TimeDelta(2.0, format='sec')
        end = event.end + TimeDelta(2.0, format='sec')
        plotAz = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                                    ['*'],  start, end)
        plotEl = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                                    ['*'],  start, end)    
        
        # Now calculates the spline fit and differentiate it to get the acceleration and jerk
        azPs = plotAz['actualPosition'].values
        azVs = plotAz['actualVelocity'].values
        azXs = plotAz['timestamp'].values - plotAz['timestamp'].values[0]  
        elPs = plotEl['actualPosition'].values
        elVs = plotEl['actualVelocity'].values
        elXs = plotEl['timestamp'].values - plotEl['timestamp'].values[0]
        plotStart = azXs[0] + 1.0
        plotEnd = azXs[-1] - 1.0
        
        deltaAz = abs(azPs[-1] - azPs[0])  
        deltaEl = abs(elPs[-1] - elPs[0]) 
        if deltaAz < 1.0 and deltaEl < 1.0:
            # eliminate very small slews
            continue
        
        plotAzXs = np.linspace(azXs[0], azXs[-1], 2000)
        plotElXs = np.linspace(elXs[0], elXs[-1], 2000)
        azPSpline = UnivariateSpline(azXs, azPs, s=0)
        azVelSpline1 = UnivariateSpline(azXs, azVs, s=0) 
        # Now smooth the derivative before differentiating again
        smoothedAzVel = np.convolve(azVelSpline1(plotAzXs), kernel, mode='same')
        azVelSpline = UnivariateSpline(plotAzXs, smoothedAzVel, s=smoothingFactor)
        azAccSpline1 = azVelSpline.derivative(n=1)
        smoothedAzAcc = np.convolve(azAccSpline1(plotAzXs), kernel, mode='same')
        # Now smooth the derivative before differentiating again
        azAccSpline = UnivariateSpline(plotAzXs, smoothedAzAcc, s=smoothingFactor)
        azJerkSpline = azAccSpline.derivative(n=1) 
        elPSpline = UnivariateSpline(elXs, elPs, s=0)
        elVelSpline1 = UnivariateSpline(elXs, elVs, s=0)
        # Now smooth the derivative before differentiating again
        smoothedElVel = np.convolve(elVelSpline1(plotElXs), kernel, mode='same')
        elVelSpline = UnivariateSpline(plotElXs, smoothedElVel, s=smoothingFactor)
        elAccSpline1 = elVelSpline.derivative(n=1)
        smoothedElAcc = np.convolve(elAccSpline1(plotElXs), kernel, mode='same')
        # Now smooth the derivative before differentiating again
        elAccSpline = UnivariateSpline(plotElXs, smoothedElAcc, s=smoothingFactor)
        elJerkSpline = elAccSpline.derivative(n=1) 
    
        
        fig.clear()
        plt.subplots_adjust(wspace=0.3, hspace=0.5)
        plt.suptitle(f"MT Mount Slews - {dayObs} - {event.seqNum:03}", fontsize = 18)
        
        plt.subplot(4,2,1)
        plt.plot(plotAzXs, azPSpline(plotAzXs), lw=3, color='r', label='Spline fit')
        plt.scatter(azXs, azPs, marker='x', color='red', s=100, label='Measured points')
        plt.title(f"Azimuth")
        plt.ylabel("Degrees")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(4,2,2)
        plt.plot(plotElXs, elPSpline(plotElXs), lw=3, color='g', label='Spline fit')
        plt.scatter(elXs, elPs, marker='x', color='g', s=100, label='Measured points')
        plt.title(f"Elevation")
        plt.ylabel("Degrees")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(4,2,3)
        plt.plot(plotAzXs, azVelSpline(plotAzXs), lw=3, color='r', label='Spline fit')
        plt.scatter(azXs, azVs, marker='x', color='red', s=100, label='Measured points')
        plt.title(f"Azimuth Velocity")
        plt.ylabel("Degrees/sec")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(4,2,4)
        plt.plot(plotElXs, elVelSpline(plotElXs), lw=3, color='g', label='Spline fit')
        plt.scatter(elXs, elVs, marker='x', color='g', s=100, label='Measured points')
        plt.title(f"Elevation Velocity")
        plt.ylabel("Degrees/sec")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(4,2,5)
        plt.plot(plotAzXs, azAccSpline(plotAzXs), lw=3, color='r', label='Acceleration')
        plt.title(f"Azimuth Acceleration")
        plt.ylabel("Degrees/sec^2")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        plt.subplot(4,2,6)
        plt.plot(plotElXs, elAccSpline(plotElXs), lw=3, color='g', label='Acceleration')
        plt.title(f"Elevation Acceleration")
        plt.ylabel("Degrees/sec^2")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        plt.subplot(4,2,7)
        plt.plot(plotAzXs, azJerkSpline(plotAzXs), lw=3, color='r', label='Jerk')
        plt.title(f"Azimuth Jerk")
        plt.ylabel("Degrees/sec^3")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        
        plt.subplot(4,2,8)
        plt.plot(plotElXs, elJerkSpline(plotElXs), lw=3, color='g', label='Jerk')
        plt.title(f"Elevation Jerk")
        plt.ylabel("Degrees/sec^3")
        plt.xlim(plotStart, plotEnd)
        plt.legend()
        plt.savefig(str(data_dir / f"MT_Mount_Accel_Jerk_{dayObs}_{event.seqNum:03}.pdf"))
        
        maxAzVel = np.max(abs(azVelSpline(plotAzXs)))
        maxAzAccel = np.max(abs(azAccSpline(plotAzXs)))
        maxAzJerk = np.max(abs(azJerkSpline(plotAzXs)))
        maxElVel = np.max(abs(elVelSpline(plotAzXs)))
        maxElAccel = np.max(abs(elAccSpline(plotAzXs)))
        maxElJerk = np.max(abs(elJerkSpline(plotAzXs)))
        maxAzVels.append(maxAzVel)
        maxAzAccels.append(maxAzAccel)
        maxAzJerks.append(maxAzJerk)
        maxElVels.append(maxElVel)
        maxElAccels.append(maxElAccel)
        maxElJerks.append(maxElJerk)
        if maxAzJerk > azLimits[2] or maxElJerk > elLimits[2] or \
        maxAzAccel > azLimits[1] or maxElAccel > elLimits[1] or \
        maxAzVel > azLimits[0] or maxElVel > elLimits[0]:
            print(f" Slew {event.seqNum} at {timestamp} exceeded limits!")
            print(f"AzVel = {maxAzVel}, AzAccel = {maxAzAccel}, AzJerk = {maxAzJerk}")
            print(f"ElVel = {maxElVel}, ElAccel = {maxElAccel}, ElJerk = {maxElJerk}")
    except:
        continue


# Now plot the Velocity/Accel/Jerk histograms.

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.subplots_adjust(wspace=0.3, hspace=0.5)
plt.suptitle(f"MT Mount Accels and Jerks", fontsize = 18)

plt.subplot(2,3,1)
plt.title("Az Max Velocity")
plt.hist(maxAzVels, color='orange')
plt.xlabel("Velocity(deg/s)")
plt.ylim(0,120)
plt.plot([azLimits[0], azLimits[0]],[0,120],color='red', ls='--', label='Max')
plt.plot([azDesign[0], azDesign[0]],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.subplot(2,3,2)
plt.title("Az Max Accels")
plt.hist(maxAzAccels, color='orange')
plt.xlabel("Accel(deg/s^2)")
plt.ylim(0,120)
plt.plot([azLimits[1], azLimits[1]],[0,120],color='red', ls='--', label='Max')
plt.plot([azDesign[1], azDesign[1]],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.subplot(2,3,3)
plt.title("Az Max Jerk")
plt.hist(maxAzJerks, color='orange')
plt.xlabel("Jerk(deg/s^3)")
plt.ylim(0,120)
plt.plot([azLimits[2], azLimits[2]],[0,120],color='red', ls='--', label='Max')
plt.plot([azDesign[2], azDesign[2]],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.subplot(2,3,4)
plt.title("El Max Velocity")
plt.hist(maxElVels, color='orange')
plt.xlabel("Velocity(deg/s)")
plt.ylim(0,120)
plt.plot([elLimits[0], elLimits[0]],[0,120],color='red', ls='--', label='Max')
plt.plot([elDesign[0], elDesign[0]],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.subplot(2,3,5)
plt.title("El Max Accels")
plt.hist(maxElAccels, color='orange')
plt.xlabel("Accel(deg/s^2)")
plt.ylim(0,120)
plt.plot([elLimits[1], elLimits[1]],[0,120],color='red', ls='--', label='Max')
plt.plot([elDesign[1], elDesign[1]],[0,120],color='blue', ls='--', label='Design')
plt.legend()
plt.subplot(2,3,6)

plt.title("El Max Jerk")
plt.hist(maxElJerks, color='orange')
plt.xlabel("Jerk(deg/s^3)")
plt.ylim(0,120)
plt.plot([elLimits[2], elLimits[2]],[0,120],color='red', ls='--', label='Max')
plt.plot([elDesign[2], elDesign[2]],[0,120],color='blue', ls='--', label='Design')
plt.legend()

plt.savefig(str(data_dir / f"Max_Accel_Jerks_{dayObs}.pdf"))